In [1]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import learning_curve
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

# `Load extracted feature maps`

In [ ]:
X_train = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/train_feature_maps.npy')
X_test = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/test_feature_maps.npy')
X_val = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/val_feature_maps.npy')
y_train = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/train_labels.npy')
y_test = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/test_labels.npy')
y_val = np.load('/kaggle/input/pit-feature-maps-mobilenetv2/val_labels.npy')

# `Convert one-hot encoded labels to class indices`

In [ ]:
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)
y_val = np.argmax(y_val, axis=1)

# `Reshape 4D feature maps to 2D (samples, features)`

In [ ]:
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)
X_val_reshaped = X_val.reshape(X_val.shape[0], -1)

# `Standardize features`

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)
X_val_scaled = scaler.transform(X_val_reshaped)

# `Apply PCA for dimensionality reduction`

In [ ]:
pca = PCA(n_components=2000)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
X_val_pca = pca.transform(X_val_scaled)

In [6]:
# Apply SMOTE to handle class imbalance
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_pca, y_train)

# `Logistic Regression`

In [7]:
logreg_model = LogisticRegression(multi_class='multinomial', max_iter=1000, solver='lbfgs', random_state=42)
logreg_model.fit(X_train_resampled, y_train_resampled)

LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42)

In [8]:
# Save model, scaler, and PCA
joblib.dump(logreg_model, "/kaggle/working/logreg.pkl")
joblib.dump(scaler, "/kaggle/working/scaler.pkl")
joblib.dump(pca, "/kaggle/working/pca.pkl")

['/kaggle/working/pca.pkl']

# `Testing performance`

In [9]:
# Predictions for each dataset
y_train_pred = logreg_model.predict(X_train_pca)
y_val_pred = logreg_model.predict(X_val_pca)
y_test_pred = logreg_model.predict(X_test_pca)

In [10]:
# Compute accuracy for train, validation, and test sets
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

In [11]:
# Print accuracies
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Training Accuracy: 85.48%
Validation Accuracy: 67.53%
Test Accuracy: 67.85%
